## CocktailDB API  
Fun little side project that generates 5 random cocktails, their ID number and all ingredients. 

In [2]:
import requests
import json
import pandas as pd

cocktail_id = [] 
names = []
ingredients = [] 

url = "https://www.thecocktaildb.com/api/json/v1/1/random.php"

# Generate 5 random cocktails that are valid
while len(cocktail_id) < 5:
    r = requests.get(url)

    if r.status_code == 200:
        site_json = r.json()

        # Check if the response contains valid data
        if 'drinks' in site_json and site_json['drinks']:
            drink = site_json['drinks'][0]
            name = drink.get('strDrink')

            # Append the columns 
            cocktail_id.append(drink['idDrink'])
            names.append(name)

            # Extract ingredients (removing Nones) -- special case
            # Keeping all arrays equal lenght; joining all ingredients into a single comma-separated list 
            ingredients_raw = [drink.get(f'strIngredient{i}', '') for i in range(1, 16)]
            ingredients.append(', '.join(filter(None, ingredients_raw)))

        else:
            continue

df_cocktail = pd.DataFrame({"Cocktail ID": cocktail_id, "Cocktail Name": names, "Ingredients":ingredients})
df_cocktail

,Cocktail ID,Cocktail Name,Ingredients
0,16134,Absolutly Screwed Up,"Absolut Citron, Orange juice, Triple sec, Ging..."
1,12560,Afterglow,"Grenadine, Orange juice, Pineapple juice"
2,11157,Brandy Cobbler,"Sugar, Club soda, Lemon, Brandy, Maraschino ch..."
3,17210,Bramble,"Gin, lemon juice, Sugar syrup, Creme de Mure"
4,12722,Strawberry Shivers,"Strawberries, Honey, Water"


Below this cell is my first attempt. I wasn't aware of two things; 
1) that there's a version of this API that is specifically made to find random cocktails.
2) That iid stands for "ingredient ID" 

Safe to say, lessons learned! Still, I'm including this one because it's part of my process and shows how my approach improved. 

In [3]:

from random import randint

ingredient_id = [] 
descriptions = []

url = "https://www.thecocktaildb.com/api/json/v1/1/lookup.php?iid="

# Generate 5 random ids that are valid
while len(ingredient_id) < 5:
    random_id = randint(1, 700) 

    r = requests.get(url + str(random_id))

    if r.status_code == 200:
        site_json = r.json()

        # Check if the response contains valid data
        if 'ingredients' in site_json and site_json['ingredients']:
            desc = site_json['ingredients'][0]['strDescription']

            if desc is not None: 
                ingredient_id.append(random_id)
                descriptions.append(desc)

        else:
            continue

# Create a DataFrame
df_ingredients = pd.DataFrame({"Cocktail ID": ingredient_id, "Description": descriptions})

df_ingredients

,Cocktail ID,Description
0,505,Schnapps (/ʃnɑːps/ or /ʃnæps/) or schnaps is a...
1,546,"Martini is a brand of Italian vermouth, named ..."
2,570,Scotch whisky (Scottish Gaelic: uisge-beatha n...
3,1,Vodka is a distilled beverage composed primari...
4,64,Schnapps (/ʃnɑːps/ or /ʃnæps/) or schnaps is a...
